In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
from ml_experiments.analyze import get_df_runs_from_mlflow_sql, get_missing_entries
from functools import partial

# Save Results

## Load mlflow runs

In [3]:
db_port = 5101
db_name = 'cohirf'
w = 'clust9'
# url = f'postgresql://{w}.ceremade.dauphine.lan:{db_port}/{db_name}'
url = f'postgresql://beluccib@localhost:{db_port}/{db_name}'
engine = create_engine(url)
query = 'SELECT experiments.name from experiments'
experiment_names = pd.read_sql(query, engine)['name'].tolist()
# results_dir = Path('~/tab_benchmark/results')
# os.makedirs(results_dir, exist_ok=True)

In [4]:
experiment_names

['Default',
 'classif_huge_cohirf',
 'classif_huge_kmeans',
 'classif_clusters_kmeans',
 'classif_clusters_cohirf',
 'classif_huge_cohirf_sampling',
 'classif_huge_cohirf_minibatch',
 'time_hc_CompleteAgglomerativeClustering_samples',
 'time_hc_WardAgglomerativeClustering_samples',
 'time_hc_OPTICS_samples',
 'time_hc_CompleteAgglomerativeClustering_features',
 'time_hc_Proclus_features',
 'time_hc_WardAgglomerativeClustering_features',
 'time_hc_KMeans_features',
 'time_hc_KMeansProj_samples',
 'time_hc_Clique_samples',
 'time_hc_DBSCAN_features',
 'time_hc_KMeans_samples',
 'time_hc_DBSCAN_samples',
 'time_hc_AffinityPropagation_samples',
 'time_hc_RecursiveClustering_features',
 'time_hc_SpectralSubspaceRandomization_samples',
 'time_hc_KMeansProj_features',
 'time_hc_MeanShift_features',
 'time_hc_HDBSCAN_features',
 'time_hc_OPTICS_features',
 'time_hc_Clique_features',
 'time_hc_IRFLLRR_features',
 'time_hc_RecursiveClustering_samples',
 'time_hc_MeanShift_samples',
 'time_hc_Spe

In [5]:
params_columns = [
    'model_nickname',
    # 'n_samples',
    # 'n_features',
    # 'n_classes',
    # 'seed_model',
    # 'dataset_id',
    # 'dataset_name',
    # 'model_params/representative_method',
    # 'model_params/n_samples_representative',
    # 'model_params/kernel_kmeans',
    # 'direction',
    # 'hpo_metric',
    # 'standardize'
]

In [6]:
# latest_metrics_columns = [
#     'fit_model_return_elapsed_time',
#     'max_memory_used',
#     'n_clusters_',
#     'rand_score',
#     'adjusted_rand',
#     'mutual_info',
#     'adjusted_mutual_info',
#     'normalized_mutual_info',
#     'homogeneity',
#     'completeness',
#     'v_measure',
#     'silhouette',
#     'calinski_harabasz_score',
#     'davies_bouldin_score',
#     'inertia_score',
#     'best_n_clusters_',
#     'best_rand_score',
#     'best_adjusted_rand',
#     'best_mutual_info',
#     'best_adjusted_mutual_info',
#     'best_normalized_mutual_info',
#     'best_homogeneity_completeness_v_measure',
#     'best_silhouette',
#     'best_calinski_harabasz_score',
#     'best_davies_bouldin_score',
#     'best_inertia_score',
#     'best_homogeneity',
#     'best_completeness',
#     'best_v_measure',
# ]

In [7]:
# tags_columns = [
#     'raised_exception',
#     'EXCEPTION',
#     'parent_run_id',
#     'best_child_run_id',
# ]

In [8]:
# # parameters of each model, they are saved as tags for the parent run
# parameters = {
#     'RecursiveClustering': ['components_size', 'repetitions', 'kmeans_n_clusters'],
#     'KMeans': ['n_clusters'],
#     'HDBSCAN': ['min_cluster_size'],
#     'DBSCAN': ['eps', 'min_samples'],
#     'AffinityPropagation': ['damping'],
#     'OPTICS': ['min_samples'],
#     'SpectralSubspaceRandomization': ['n_similarities', 'sampling_ratio', 'sc_n_clusters'],
#     'WardAgglomerativeClustering': ['n_clusters'],
#     'Clique': ['n_partitions', 'density_threshold'],
#     'IRFLLRR': ['sc_n_clusters', 'p', 'c', 'lambda_', 'alpha'],
#     'KMeansProj': ['n_clusters'],
#     'Proclus': ['n_clusters', 'avg_dims'],
#     'SpectralClustering': ['n_clusters'],
#     'SingleAgglomerativeClustering': ['n_clusters'],
#     'CompleteAgglomerativeClustering': ['n_clusters'],
#     'AverageAgglomerativeClustering': ['n_clusters'],
# }
# all_model_parameters = [p for model, params in parameters.items() for p in params]
# all_model_parameters = list(set(all_model_parameters))
# for param in all_model_parameters:
#     tags_columns.append(param)

In [10]:
runs_columns = ['run_uuid', 'status', 'start_time', 'end_time']
experiments_columns = []
other_table = 'params'
other_table_keys = params_columns
df_params = get_df_runs_from_mlflow_sql(engine, runs_columns=runs_columns, experiments_columns=experiments_columns, experiments_names=experiment_names, other_table=other_table, other_table_keys=other_table_keys)
# df_latest_metrics = get_df_runs_from_mlflow_sql(engine, runs_columns=['run_uuid'], experiments_columns=experiments_columns, experiments_names=experiments_names, other_table='latest_metrics', other_table_keys=latest_metrics_columns)
# df_tags = get_df_runs_from_mlflow_sql(engine, runs_columns=['run_uuid'], experiments_columns=experiments_columns, experiments_names=experiments_names, other_table='tags', other_table_keys=tags_columns)

In [12]:
df = df_params.copy()
df = df.loc[df['model_nickname']=='Clique']

In [13]:
df

key,status,start_time,end_time,model_nickname
run_uuid,,,,
0016d7a9f30f4e7cb4f47a25912a5cf4,RUNNING,1742842775239,NaN,Clique
00260d9d17c248d09c64c326b4c217c1,FINISHED,1742901019627,1.742901e+12,Clique
0036bfe07f394388a564532f52ff41b8,FAILED,1742938518449,1.742939e+12,Clique
0052583b7d5649ed8e5bcd2509e79f59,FINISHED,1742900983940,1.742901e+12,Clique
00ed0a521a894f4498228cd4c7839b70,FAILED,1742922360629,1.742930e+12,Clique
...,...,...,...,...
fea2200b8b224267adf267f7999d55a9,FAILED,1742938444040,1.742938e+12,Clique
feaaae16d03d4164827c0ed00312428f,FAILED,1742938518564,1.742939e+12,Clique
fec24c2dea5e49159a753392892c448a,FAILED,1742938445471,1.742939e+12,Clique


In [15]:
run_ids = list(df.index)

In [16]:
runs_to_delete = run_ids

In [17]:
runs_to_delete

['0016d7a9f30f4e7cb4f47a25912a5cf4',
 '00260d9d17c248d09c64c326b4c217c1',
 '0036bfe07f394388a564532f52ff41b8',
 '0052583b7d5649ed8e5bcd2509e79f59',
 '00ed0a521a894f4498228cd4c7839b70',
 '00f21f610197484fb3055e681edf9bd6',
 '0170fd0f401d4087aaeaa635f15cc326',
 '028b8fad603e4a0fa214bcb0b16a049e',
 '02ae728d69784922a45626c78637f779',
 '02bcea928a174ac89865c246339ebeaf',
 '02cbdbee3acc430b8f2e7e2bbc82d607',
 '02e7d5b2c96c41bfac4766f41030e96d',
 '03ca02bde0bc426fbbabe412bd700b7d',
 '041eb63399a44aa6b461b842e7f4b701',
 '04a7b235d0fd486e9bc8bff089099ac6',
 '04d1dcb42631418d82199d062665d3a3',
 '051e7546218040b28f6ab682ccdc324b',
 '058eba7c7c8b417c81f9f78fc7bceb0f',
 '058f93366fc544aa8667a59dadf3f374',
 '0674712bbba940da81f957521fc7f6d0',
 '067c02a61e724a5595c38ff8da9fee99',
 '069038a295be48f68fe8b9829e16056d',
 '077db6dfa296441a83393a1d42e6bd20',
 '07d250b5d1a644a097a694f4224de8c0',
 '084875a8fd1a410aa2b84c1993fa448e',
 '08afb980ad864eb9a8c92eb1f895a6c3',
 '08ccd58e2fea4449be0de1e6ac32d1cb',
 

In [18]:
run_uuid_query = [f"'{run_id}'" for run_id in runs_to_delete]
run_uuid_query = ', '.join(run_uuid_query)

In [19]:
query = f"""
UPDATE runs
SET lifecycle_stage = 'deleted'
WHERE run_uuid IN ({run_uuid_query}) 
"""
with engine.begin() as conn:
    conn.execute(text(query))

In [20]:
query = f"""
DELETE
FROM
	experiment_tags
WHERE
	experiment_id = ANY(
	SELECT
		experiment_id
	FROM
		experiments
	WHERE
		lifecycle_stage = 'deleted');

DELETE
FROM
	latest_metrics
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');

DELETE
FROM
	metrics
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');

DELETE
FROM
	params
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');

DELETE
FROM
	tags
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');

DELETE
FROM
	runs
WHERE
	lifecycle_stage = 'deleted';

DELETE
FROM
	experiments
WHERE
	lifecycle_stage = 'deleted';
"""

with engine.begin() as conn:
    conn.execute(text(query))